In [44]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd

def fetch_box_score():
    hrefs = set()
    url = "https://www.basketball-reference.com/playoffs/NBA_2023.html"

    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Steps #1 to #9
    table_container = soup.find("div", {"id": "div_all_playoffs"})
    tbody = table_container.find("tbody")
    tr_toggleable = tbody.find_all("tr", {"class": "toggleable"})
    for trs in tr_toggleable:
        a_s = trs.find_all("a")
        [hrefs.add(x["href"]) for x in a_s]


    for href_link in list(hrefs):
        boxscore_link = f"https://www.basketball-reference.com{href_link}"
        print(boxscore_link)
        response = requests.get(boxscore_link)
        soup = BeautifulSoup(response.content, 'html.parser')
        titles = set()
        tags_with_text = soup.find_all(lambda tag: " and Advanced Stats" in tag.text)
        lowest_level_tags = [tag for tag in tags_with_text if
                             not tag.find(lambda child: " and Advanced Stats" in child.text)]
        for x in lowest_level_tags:
            titles.add(x.text.replace("Table", "").replace("Basic and Advanced Stats","").strip())
        print(titles)

        rows = []
        soup = soup.find_all(lambda tag: tag.has_attr('class') and any('-game' in cls for cls in tag['class']))
        count = 0
        for x in soup:
            for tbody in x.find_all("tbody"):
                isStarter = True
                for trs in tbody.find_all("tr"):
                    row = {}
                    if count < 2:
                        row["team_name"] = list(titles)[0]
                    else:
                        row["team_name"] = list(titles)[1]
                        
                    row["name"] = trs.find("th").text
                    row["starter"] = isStarter
                    if trs.find("th").text == "Reserves":
                        isStarter = False
                        continue
                    for tds in trs.find_all("td"):
                        row[tds["data-stat"]] = tds.text
                    rows.append(row)
                count +=1
                
                
        pd.DataFrame(rows)        
        basic_column_names = ["team_name","starter",'name', 'mp', 'fg', 'fga', 'fg_pct', 'fg3', 'fg3a', 'fg3_pct', 'ft','fta', 'ft_pct', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf','pts', 'plus_minus']
        df_basic = df[basic_column_names].dropna()
        df_basic.to_csv(list(titles)[0]+ " vs " + list(titles)[1] + " basic stats", index=False)
        
fetch_box_score()


https://www.basketball-reference.com/boxscores/202304260MIL.html
{'Miami Heat', 'Milwaukee Bucks'}
https://www.basketball-reference.com/boxscores/202304150PHI.html
{'Philadelphia 76ers', 'Brooklyn Nets'}
https://www.basketball-reference.com/boxscores/202306010DEN.html
{'Denver Nuggets', 'Miami Heat'}
https://www.basketball-reference.com/boxscores/202304160MEM.html
{'Memphis Grizzlies', 'Los Angeles Lakers'}
https://www.basketball-reference.com/boxscores/202305050PHO.html
{'Denver Nuggets', 'Phoenix Suns'}
https://www.basketball-reference.com/boxscores/202305070PHO.html
{'Denver Nuggets', 'Phoenix Suns'}
https://www.basketball-reference.com/boxscores/202304150CLE.html
{'Cleveland Cavaliers', 'New York Knicks'}
https://www.basketball-reference.com/boxscores/202304210NYK.html
{'Cleveland Cavaliers', 'New York Knicks'}
https://www.basketball-reference.com/boxscores/202305090BOS.html
{'Philadelphia 76ers', 'Boston Celtics'}
https://www.basketball-reference.com/boxscores/202304230ATL.html
{'

KeyboardInterrupt: 

,team_name,starter,name,mp,fg,fga,fg_pct,fg3,fg3a,fg3_pct,...,orb,drb,trb,ast,stl,blk,tov,pf,pts,plus_minus
0,Denver Nuggets,True,Karl-Anthony Towns,29:34,5,15,.333,1,7,.143,...,1,9,10,2,0,0,4,2,11,-11
1,Denver Nuggets,True,Anthony Edwards,27:41,6,15,.400,1,4,.250,...,0,2,2,5,2,1,1,4,18,-12
2,Denver Nuggets,True,Rudy Gobert,25:43,3,5,.600,0,0,,...,2,11,13,0,0,2,1,3,8,-28
3,Denver Nuggets,True,Mike Conley,24:57,3,5,.600,2,3,.667,...,0,4,4,3,0,0,0,0,8,-18
4,Denver Nuggets,True,Nickeil Alexander-Walker,23:30,2,6,.333,2,4,.500,...,0,1,1,0,0,1,0,1,7,-6
5,Denver Nuggets,False,Kyle Anderson,25:57,5,11,.455,1,3,.333,...,0,1,1,3,2,1,0,1,11,-15
6,Denver Nuggets,False,Jaylen Nowell,21:54,4,10,.400,3,6,.500,...,1,0,1,2,0,0,1,1,12,-13
7,Denver Nuggets,False,Taurean Prince,21:17,0,4,.000,0,3,.000,...,0,0,0,2,0,0,0,1,0,-14
8,Denver Nuggets,False,Austin Rivers,16:12,1,2,.500,1,2,.500,...,0,3,3,0,0,0,1,0,3,-12
9,Denver Nuggets,False,Jordan McLaughlin,10:19,0,3,.000,0,1,.000,...,0,2,2,1,0,0,0,2,0,-8


In [4]:
df_selected.to_csv('output.csv', index=False)